In [1]:
!pip install transformers peft torch accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
import gdown
import os

os.makedirs("model_files", exist_ok=True)
os.chdir("model_files")

print("Downloading your trained model files from Google Drive...")

folder_url = "https://drive.google.com/drive/folders/1AKzzA2WObNmontbDVaAY8ZPNF7g669Mr"
gdown.download_folder(folder_url, quiet=False, use_cookies=False)

Retrieving folder contents


Processing file 1ufkk-Pok1Jj27YxjWSmD92pXrC5DKv1q adapter_config.json
Processing file 1TyESkqVSppq_w1MROrBxQvXghQSNtWJ1 adapter_model.safetensors
Processing file 1CvMpQvyBcuzhD7uD-stCtNxMZuo1ufzp sample_training_data.json
Processing file 1CxtjWrFArCUY2-Jnn_QinsX_tOYoPyAD special_tokens_map.json
Processing file 1SEt3zkMfSpB969JwEbJhTmefGRNp5Iij tokenizer_config.json
Processing file 16l-plbzWhdXhJSPB1eSvnD7YS-vsSkrZ tokenizer.json


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1ufkk-Pok1Jj27YxjWSmD92pXrC5DKv1q
To: /content/model_files/QA_finetuning_test/adapter_config.json
100%|██████████| 765/765 [00:00<00:00, 2.59MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1TyESkqVSppq_w1MROrBxQvXghQSNtWJ1
From (redirected): https://drive.google.com/uc?id=1TyESkqVSppq_w1MROrBxQvXghQSNtWJ1&confirm=t&uuid=2009feee-7423-401c-9e05-8db4a013ed69
To: /content/model_files/QA_finetuning_test/adapter_model.safetensors
100%|██████████| 446M/446M [00:08<00:00, 51.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CvMpQvyBcuzhD7uD-stCtNxMZuo1ufzp
To: /content/model_files/QA_finetuning_test/sample_training_data.json
100%|██████████| 10.4k/10.4k [00:00<00:00, 30.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CxtjWrFArCUY2-Jnn_QinsX_tOYoPyAD
To: /content/model_files/QA_finetuning_test/spe

['/content/model_files/QA_finetuning_test/adapter_config.json',
 '/content/model_files/QA_finetuning_test/adapter_model.safetensors',
 '/content/model_files/QA_finetuning_test/sample_training_data.json',
 '/content/model_files/QA_finetuning_test/special_tokens_map.json',
 '/content/model_files/QA_finetuning_test/tokenizer_config.json',
 '/content/model_files/QA_finetuning_test/tokenizer.json']

In [3]:
print("\nDownloaded files:")
for root, dirs, files in os.walk("."):
    for file in files:
        filepath = os.path.join(root, file)
        size = os.path.getsize(filepath) / (1024*1024)  # Size in MB
        print(f"  {filepath}: {size:.1f} MB")


Downloaded files:
  ./QA_finetuning_test/sample_training_data.json: 0.0 MB
  ./QA_finetuning_test/tokenizer_config.json: 0.0 MB
  ./QA_finetuning_test/tokenizer.json: 6.8 MB
  ./QA_finetuning_test/adapter_model.safetensors: 425.0 MB
  ./QA_finetuning_test/adapter_config.json: 0.0 MB
  ./QA_finetuning_test/special_tokens_map.json: 0.0 MB


In [4]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU device: Tesla T4


In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import json
import random

def load_model_with_fallback():
    """Load your fine-tuned model with fallback options"""
    base_model_name = "microsoft/Phi-4-reasoning-plus"
    adapter_path = "./QA_finetuning_test/"

    print(f"Loading from adapter path: {adapter_path}")

    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)

    print("Loading base model...")
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )

    try:
        # Method 1: Standard loading
        print("  Trying standard PEFT loading...")
        model = PeftModel.from_pretrained(base_model, adapter_path)
        print("Standard loading successful!")
        return model, tokenizer

    except Exception as e1:
        print(f"  Standard loading failed: {str(e1)[:100]}...")

        try:
            # Method 2: Load with specific config
            print("  Trying with explicit config...")
            peft_config = PeftConfig.from_pretrained(adapter_path)
            model = PeftModel.from_pretrained(
                base_model,
                adapter_path,
                config=peft_config,
                is_trainable=False
            )
            print("Config-based loading successful!")
            return model, tokenizer

        except Exception as e2:
            print(f"  Config loading failed: {str(e2)[:100]}...")

            try:
                # Method 3: Force loading with adapter_name
                print("  Trying with default adapter name...")
                model = PeftModel.from_pretrained(
                    base_model,
                    adapter_path,
                    adapter_name="default"
                )
                print("Named adapter loading successful!")
                return model, tokenizer

            except Exception as e3:
                print(f"  Named loading failed: {str(e3)[:100]}...")

                # Method 4: Just use base model (no fine-tuning)
                print("  Using base model without fine-tuning...")
                print("  This will test the base Phi-4 model, not your trained version")
                return base_model, tokenizer

def inspect_adapter_files():
    adapter_path = "./QA_finetuning_test/"

    print("\n Inspecting adapter files:")

    try:
        with open(f"{adapter_path}/adapter_config.json", 'r') as f:
            config = json.load(f)
        print(f" Adapter config loaded:")
        for key, value in config.items():
            print(f"  {key}: {value}")
    except Exception as e:
        print(f" Error reading adapter config: {e}")

    import os
    print(f"\n📁 Files in adapter directory:")
    for file in os.listdir(adapter_path):
        size = os.path.getsize(f"{adapter_path}/{file}") / (1024*1024)
        print(f"  {file}: {size:.1f} MB")

inspect_adapter_files()

print("\n Loading your fine-tuned Phi-4 model...")
try:
    model, tokenizer = load_model_with_fallback()
    model_loaded = True
    print(" Model loading completed!")
except Exception as e:
    print(f"All loading methods failed: {e}")
    model_loaded = False



 Inspecting adapter files:
 Adapter config loaded:
  alpha_pattern: {}
  auto_mapping: None
  base_model_name_or_path: microsoft/Phi-4-reasoning-plus
  bias: none
  eva_config: None
  exclude_modules: None
  fan_in_fan_out: None
  inference_mode: True
  init_lora_weights: True
  layer_replication: None
  layers_pattern: None
  layers_to_transform: None
  loftq_config: {}
  lora_alpha: 16
  lora_bias: False
  lora_dropout: 0.05
  megatron_config: None
  megatron_core: megatron.core
  modules_to_save: None
  peft_type: LORA
  r: 32
  rank_pattern: {}
  revision: None
  target_modules: ['o_proj', 'down_proj', 'qkv_proj', 'gate_up_proj']
  task_type: CAUSAL_LM
  use_dora: False
  use_rslora: False

📁 Files in adapter directory:
  sample_training_data.json: 0.0 MB
  tokenizer_config.json: 0.0 MB
  tokenizer.json: 6.8 MB
  adapter_model.safetensors: 425.0 MB
  adapter_config.json: 0.0 MB
  special_tokens_map.json: 0.0 MB

 Loading your fine-tuned Phi-4 model...
Loading from adapter path: ./

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for base_model.model.model.layers.31.self_attn.o_proj.lora_A.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for base_model.model.model.layers.31.self_attn.o_proj.lora_B.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign

  Config loading failed: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instea...
  Trying with default adapter name...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for base_model.model.model.layers.31.self_attn.o_proj.lora_A.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for base_model.model.model.layers.31.self_attn.o_proj.lora_B.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for base_model.model.model

  Named loading failed: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instea...
  Using base model without fine-tuning...
  This will test the base Phi-4 model, not your trained version
 Model loading completed!


/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 256 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


In [6]:

def load_questions_from_sample_data():
    sample_file = "./QA_finetuning_test/sample_training_data.json"

    print(f" Loading questions from {sample_file}...")

    with open(sample_file, 'r') as f:
        data = json.load(f)

    print(f"Found {len(data)} conversations in sample data")

    questions = []

    for i, conversation in enumerate(data):
        messages = conversation['messages']

        for j, message in enumerate(messages):
            if (message['role'] == 'user' and
                len(message['content']) > 100 and
                'Consider the following constraints' in message['content']):

                original_answer = None
                if j + 1 < len(messages) and messages[j + 1]['role'] == 'assistant':
                    original_answer = messages[j + 1]['content']

                questions.append({
                    'conversation_id': i,
                    'question': message['content'],
                    'original_answer': original_answer
                })
                break

    print(f" Extracted {len(questions)} questions from sample training data")
    return questions

if model_loaded:
    test_questions = load_questions_from_sample_data()

def generate_answer(model, tokenizer, question):
    """Generate answer for a given question"""
    prompt = f"""<|im_start|>system
You are an expert in symbolic computation and polynomial decomposition.
Your task is to help rewrite a target polynomial into the required form based on given inequality premises.
<|im_end|>
<|im_start|>user
{question}
<|im_end|>
<|im_start|>assistant
"""

    inputs = tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,  # Reduced for faster generation
            temperature=0.3,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = response.split("<|im_start|>assistant")[-1].strip()
    return answer

if not model_loaded:
    print("Cannot run tests - model failed to load")
else:
    results = []

    num_tests = min(2, len(test_questions))
    test_subset = test_questions[:num_tests]

    print(f"\n Testing model with {num_tests} questions...")
    print("=" * 80)

    for i, q_data in enumerate(test_subset, 1):
        question = q_data['question']
        original_answer = q_data['original_answer']

        print(f"\n QUESTION {i}:")
        print(f"{question[:150]}...")

        print(f"\n GENERATING ANSWER...")

        try:
            generated_answer = generate_answer(model, tokenizer, question)

            print(f"GENERATED: {generated_answer}")
            print(f" ORIGINAL:  {original_answer[:150]}..." if original_answer else "N/A")
            print("-" * 80)

            results.append({
                "question_id": i,
                "question": question,
                "generated_answer": generated_answer,
                "original_answer": original_answer,
                "status": "success"
            })

        except Exception as e:
            print(f"Generation error: {str(e)}")
            print("-" * 80)

            results.append({
                "question_id": i,
                "question": question,
                "generated_answer": f"Error: {str(e)}",
                "original_answer": original_answer,
                "status": "error"
            })

    if results:
        with open('/content/adapter_test_results.json', 'w') as f:
            json.dump(results, f, indent=2)

        successful_results = [r for r in results if r['status'] == 'success']
        print(f"\n🎉 Testing complete!")
        print(f"Success rate: {len(successful_results)}/{len(results)}")

        from google.colab import files
        files.download('/content/adapter_test_results.json')

 Loading questions from ./QA_finetuning_test/sample_training_data.json...


JSONDecodeError: Expecting property name enclosed in double quotes: line 101 column 1 (char 10383)